# KNN, K-Means & Clustering from Scratch

**Welcome, St. Mark!** Now we explore distance-based methods and unsupervised learning. Think of this as understanding how similar patients cluster together and how we can classify based on proximity.

We'll explore:

1. **K-Nearest Neighbors (KNN)** - Classification by proximity
2. **K-Means Clustering** - Lloyd's algorithm for grouping data
3. **Hierarchical Clustering** - Agglomerative clustering methods
4. **Distance Metrics** - Euclidean, Manhattan, and cosine distances
5. **Cluster Evaluation** - Silhouette scores and elbow method

By the end, you'll understand how proximity drives both supervised and unsupervised learning.

## The Big Picture

Distance-based methods find patterns through proximity:
- **KNN:** Classify based on nearest neighbors' labels
- **K-Means:** Group data into K clusters by minimizing within-cluster distances
- **Hierarchical:** Build nested clusters through progressive merging
- **Distance Metrics:** Different ways to measure "closeness"

**Key Question:** How do we leverage proximity for both classification and clustering?

## Data Preparation: Clustering-Friendly Dataset

We'll use scikit-learn's `make_blobs` to create clear clusters for demonstration.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import accuracy_score, silhouette_score, classification_report
from sklearn.datasets import make_blobs, make_classification
from sklearn.model_selection import train_test_split
from collections import Counter
import time

# Create clustering dataset with clear groups
X_clusters, y_clusters = make_blobs(n_samples=300,
                                   centers=4,
                                   cluster_std=1.5,
                                   random_state=42)

# Create classification dataset for KNN
X_clf, y_clf = make_classification(n_samples=200,
                                  n_features=2,
                                  n_classes=3,
                                  n_informative=2,
                                  n_redundant=0,
                                  n_clusters_per_class=1,
                                  random_state=42)

# Split classification data
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.3, random_state=42)

print(f"Clustering dataset: {X_clusters.shape[0]} samples, {len(np.unique(y_clusters))} true clusters")
print(f"Classification dataset: {X_train_clf.shape[0]} train, {X_test_clf.shape[0]} test samples")
print(f"Feature ranges - Clustering: [{X_clusters.min():.2f}, {X_clusters.max():.2f}]")
print(f"Feature ranges - Classification: [{X_clf.min():.2f}, {X_clf.max():.2f}]")

**Cell Analysis:** We've prepared two datasets.

- **Clustering data:** Clear groups for unsupervised learning
- **Classification data:** Labeled data for KNN supervised learning
- **Different scales:** Shows algorithms handle various data ranges

**Healthcare Analogy:** Like analyzing patient groups - some data has known diagnoses (supervised), other data needs grouping discovery (unsupervised).

**Reflection Question:** When would you use clustering vs classification in medical data analysis?

## Method 1: Distance Metrics - Measuring Proximity

Different ways to measure how "close" two points are:

- **Euclidean:** Straight-line distance (most common)
- **Manhattan:** City block distance (taxicab metric)
- **Cosine:** Angle-based similarity (normalized direction)

In [ ]:
def euclidean_distance(a, b):
    """Euclidean (L2) distance between two points."""
    return np.sqrt(np.sum((a - b) ** 2))

def manhattan_distance(a, b):
    """Manhattan (L1) distance between two points."""
    return np.sum(np.abs(a - b))

def cosine_similarity(a, b):
    """Cosine similarity (converted to distance)."""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    similarity = dot_product / (norm_a * norm_b)
    # Convert similarity to distance (1 - similarity)
    return 1 - similarity

def compute_distance_matrix(X, metric='euclidean'):
    """Compute pairwise distance matrix for all points."""
    n_samples = X.shape[0]
    distance_matrix = np.zeros((n_samples, n_samples))
    
    for i in range(n_samples):
        for j in range(i + 1, n_samples):
            if metric == 'euclidean':
                dist = euclidean_distance(X[i], X[j])
            elif metric == 'manhattan':
                dist = manhattan_distance(X[i], X[j])
            elif metric == 'cosine':
                dist = cosine_similarity(X[i], X[j])
            else:
                raise ValueError(f"Unknown metric: {metric}")
            
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # Symmetric
    
    return distance_matrix

# Test distance metrics on sample points
point_a = np.array([1, 2])
point_b = np.array([4, 6])
point_c = np.array([1, 6])  # Same x as A, same y as B

points = [point_a, point_b, point_c]
point_names = ['A(1,2)', 'B(4,6)', 'C(1,6)']

print("Distance Metrics Comparison:")
print("=" * 50)
print(f"{'Points':<12} {'Euclidean':<10} {'Manhattan':<10} {'Cosine':<10}")
print("-" * 50)

for i in range(len(points)):
    for j in range(i + 1, len(points)):
        eucl = euclidean_distance(points[i], points[j])
        manh = manhattan_distance(points[i], points[j])
        cos = cosine_similarity(points[i], points[j])
        print(f"{point_names[i]}→{point_names[j]:<6} {eucl:<10.3f} {manh:<10.3f} {cos:<10.3f}")

# Visualize distance metrics
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot points
for ax in axes:
    ax.scatter([p[0] for p in points], [p[1] for p in points], s=100, c='red')
    for i, name in enumerate(point_names):
        ax.annotate(name, (points[i][0] + 0.1, points[i][1] + 0.1), fontsize=12)
    ax.set_xlim(0, 5)
    ax.set_ylim(0, 7)
    ax.grid(True, alpha=0.3)

# Euclidean circles
axes[0].set_title('Euclidean Distance\n(Straight-line)')
circle_a = plt.Circle((1, 2), euclidean_distance(point_a, point_b), fill=False, color='blue', linestyle='--')
axes[0].add_patch(circle_a)

# Manhattan diamonds
axes[1].set_title('Manhattan Distance\n(City blocks)')
diamond_points = np.array([[1, 2 + manhattan_distance(point_a, point_b)],
                          [1 + manhattan_distance(point_a, point_b), 2],
                          [1, 2 - manhattan_distance(point_a, point_b)],
                          [1 - manhattan_distance(point_a, point_b), 2]])
axes[1].plot(diamond_points[:, 0], diamond_points[:, 1], 'g--')

# Cosine angles
axes[2].set_title('Cosine Distance\n(Angle-based)')
# Draw vectors from origin
for point, name in zip(points, point_names):
    axes[2].arrow(0, 0, point[0], point[1], head_width=0.1, head_length=0.1, 
                  fc='blue', ec='blue', alpha=0.7)
    axes[2].text(point[0] + 0.1, point[1] + 0.1, name, fontsize=10)

plt.tight_layout()
plt.show()

**Cell Analysis:**

**Distance Metrics:**
- **Euclidean:** Straight-line distance, sensitive to all dimensions equally
- **Manhattan:** Grid-based distance, good for categorical features
- **Cosine:** Direction-based, ignores magnitude, good for text/high-dim data

**Healthcare Analogy:** Different ways doctors measure patient similarity - symptoms, demographics, or treatment response patterns.

**Reflection Question:** Which distance metric would you choose for comparing patient symptoms vs genetic profiles?

## Method 2: K-Nearest Neighbors (KNN) Classification

KNN classifies by majority vote of K nearest neighbors:

1. Find K closest training points to test point
2. Let neighbors "vote" on the class
3. Assign most common class as prediction

In [ ]:
class KNNClassifier:
    """
    K-Nearest Neighbors classifier from scratch.
    """
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None
        self.y_train = None
    
    def fit(self, X, y):
        """Store training data."""
        self.X_train = X
        self.y_train = y
    
    def _compute_distance(self, a, b):
        """Compute distance between two points."""
        if self.distance_metric == 'euclidean':
            return euclidean_distance(a, b)
        elif self.distance_metric == 'manhattan':
            return manhattan_distance(a, b)
        elif self.distance_metric == 'cosine':
            return cosine_similarity(a, b)
        else:
            raise ValueError(f"Unknown distance metric: {self.distance_metric}")
    
    def predict(self, X):
        """Predict classes for test data."""
        predictions = []
        
        for x_test in X:
            # Compute distances to all training points
            distances = [self._compute_distance(x_test, x_train) 
                        for x_train in self.X_train]
            
            # Get indices of k nearest neighbors
            k_nearest_indices = np.argsort(distances)[:self.k]
            
            # Get labels of k nearest neighbors
            k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]
            
            # Majority vote
            majority_vote = Counter(k_nearest_labels).most_common(1)[0][0]
            predictions.append(majority_vote)
        
        return np.array(predictions)

# Train and test our KNN
knn = KNNClassifier(k=3, distance_metric='euclidean')
knn.fit(X_train_clf, y_train_clf)

knn_predictions = knn.predict(X_test_clf)
knn_accuracy = accuracy_score(y_test_clf, knn_predictions)

print(f"Our KNN (k=3) Test Accuracy: {knn_accuracy:.4f}")

# Compare with scikit-learn
sk_knn = KNeighborsClassifier(n_neighbors=3)
sk_knn.fit(X_train_clf, y_train_clf)
sk_predictions = sk_knn.predict(X_test_clf)
sk_accuracy = accuracy_score(y_test_clf, sk_predictions)

print(f"Scikit-learn KNN Test Accuracy: {sk_accuracy:.4f}")
print(f"Accuracy difference: {abs(knn_accuracy - sk_accuracy):.4f}")

# Test different k values
k_values = [1, 3, 5, 7, 9]
accuracies = []

for k in k_values:
    knn_temp = KNNClassifier(k=k)
    knn_temp.fit(X_train_clf, y_train_clf)
    pred_temp = knn_temp.predict(X_test_clf)
    acc_temp = accuracy_score(y_test_clf, pred_temp)
    accuracies.append(acc_temp)
    print(f"K={k}: Test Accuracy = {acc_temp:.4f}")

# Plot k vs accuracy
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, 'bo-', linewidth=2, markersize=8)
plt.xlabel('K (Number of Neighbors)')
plt.ylabel('Test Accuracy')
plt.title('KNN: Effect of K on Classification Accuracy')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.show()

**Cell Analysis:**

**KNN Process:**
- **Distance computation:** Find closest training points
- **Neighbor selection:** Choose K most similar examples
- **Majority voting:** Let similar examples decide the class

**K Selection Trade-offs:**
- **Small K:** Sensitive to noise, may overfit
- **Large K:** Smoother decisions, may underfit

**Healthcare Analogy:** Like consulting K similar patient cases to diagnose a new patient.

**Reflection Question:** How does KNN's "lazy learning" differ from the "eager learning" of logistic regression?

## Method 3: K-Means Clustering - Lloyd's Algorithm

K-Means groups data into K clusters by minimizing within-cluster distances:

1. **Initialize:** Randomly place K cluster centers
2. **Assign:** Each point belongs to nearest center
3. **Update:** Move centers to mean of assigned points
4. **Repeat:** Until centers stop moving

In [ ]:
class KMeansClustering:
    """
    K-Means clustering using Lloyd's algorithm.
    """
    def __init__(self, n_clusters=3, max_iter=100, random_state=42):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state
        self.centers = None
        self.labels = None
    
    def fit(self, X):
        """Fit K-Means clustering."""
        np.random.seed(self.random_state)
        n_samples, n_features = X.shape
        
        # Initialize centers randomly from data points
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centers = X[random_indices].copy()
        
        for iteration in range(self.max_iter):
            # Step 1: Assign each point to nearest center
            distances = np.zeros((n_samples, self.n_clusters))
            for i in range(n_samples):
                for j in range(self.n_clusters):
                    distances[i, j] = euclidean_distance(X[i], self.centers[j])
            
            self.labels = np.argmin(distances, axis=1)
            
            # Step 2: Update centers to mean of assigned points
            new_centers = np.zeros_like(self.centers)
            for cluster in range(self.n_clusters):
                cluster_points = X[self.labels == cluster]
                if len(cluster_points) > 0:
                    new_centers[cluster] = np.mean(cluster_points, axis=0)
                else:
                    # Reinitialize empty clusters
                    new_centers[cluster] = X[np.random.choice(n_samples)]
            
            # Check for convergence
            center_shift = np.sum(np.abs(new_centers - self.centers))
            self.centers = new_centers
            
            if center_shift < 1e-6:
                print(f"Converged after {iteration + 1} iterations")
                break
        
        return self
    
    def predict(self, X):
        """Predict cluster labels for new data."""
        n_samples = X.shape[0]
        distances = np.zeros((n_samples, self.n_clusters))
        
        for i in range(n_samples):
            for j in range(self.n_clusters):
                distances[i, j] = euclidean_distance(X[i], self.centers[j])
        
        return np.argmin(distances, axis=1)

# Apply our K-Means to clustering data
kmeans = KMeansClustering(n_clusters=4, random_state=42)
kmeans.fit(X_clusters)

print(f"Final cluster centers:")
for i, center in enumerate(kmeans.centers):
    print(f"Cluster {i}: ({center[0]:.2f}, {center[1]:.2f})")

# Compare with scikit-learn
sk_kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
sk_labels = sk_kmeans.fit_predict(X_clusters)

print(f"\nOur K-Means silhouette score: {silhouette_score(X_clusters, kmeans.labels):.4f}")
print(f"Scikit-learn silhouette score: {silhouette_score(X_clusters, sk_labels):.4f}")

# Visualize clustering results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Our implementation
scatter1 = ax1.scatter(X_clusters[:, 0], X_clusters[:, 1], c=kmeans.labels, 
                      cmap='viridis', alpha=0.6, s=50)
ax1.scatter(kmeans.centers[:, 0], kmeans.centers[:, 1], c='red', 
           marker='x', s=200, linewidth=3, label='Centers')
ax1.set_title('Our K-Means Clustering')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Scikit-learn
scatter2 = ax2.scatter(X_clusters[:, 0], X_clusters[:, 1], c=sk_labels, 
                      cmap='viridis', alpha=0.6, s=50)
ax2.scatter(sk_kmeans.cluster_centers_[:, 0], sk_kmeans.cluster_centers_[:, 1], 
           c='red', marker='x', s=200, linewidth=3, label='Centers')
ax2.set_title('Scikit-learn K-Means')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Cell Analysis:**

**K-Means Process:**
- **Expectation:** Assign points to nearest centers
- **Maximization:** Update centers to cluster means
- **Convergence:** Iterate until centers stabilize

**Algorithm Properties:**
- **Sensitive to initialization:** Random starts can give different results
- **Assumes spherical clusters:** Works best with round, similar-sized groups
- **Hard assignments:** Each point belongs to exactly one cluster

**Healthcare Analogy:** Like grouping patients by symptom patterns - malaria patients cluster differently from typhoid patients.

**Reflection Question:** Why might K-Means fail on elongated or irregular-shaped clusters?

## Method 4: Hierarchical Clustering - Agglomerative Method

Hierarchical clustering builds a tree of nested clusters:

1. **Start:** Each point is its own cluster
2. **Merge:** Combine closest clusters iteratively
3. **Stop:** When desired number of clusters reached

**Linkage Methods:** Single, Complete, Average, Ward

In [ ]:
class HierarchicalClustering:
    """
    Agglomerative hierarchical clustering.
    """
    def __init__(self, n_clusters=3, linkage='single'):
        self.n_clusters = n_clusters
        self.linkage = linkage  # 'single', 'complete', 'average'
        self.labels = None
    
    def fit(self, X):
        """Perform agglomerative clustering."""
        n_samples = X.shape[0]
        
        # Initialize: each point is its own cluster
        clusters = [[i] for i in range(n_samples)]
        
        # Compute initial distance matrix
        distance_matrix = compute_distance_matrix(X)
        
        while len(clusters) > self.n_clusters:
            # Find closest pair of clusters
            min_distance = float('inf')
            merge_i, merge_j = -1, -1
            
            for i in range(len(clusters)):
                for j in range(i + 1, len(clusters)):
                    # Compute distance between clusters based on linkage
                    cluster_dist = self._cluster_distance(clusters[i], clusters[j], 
                                                        distance_matrix)
                    
                    if cluster_dist < min_distance:
                        min_distance = cluster_dist
                        merge_i, merge_j = i, j
            
            # Merge the closest clusters
            clusters[merge_i].extend(clusters[merge_j])
            del clusters[merge_j]
        
        # Convert cluster lists to labels
        self.labels = np.zeros(n_samples, dtype=int)
        for cluster_id, cluster in enumerate(clusters):
            for point_idx in cluster:
                self.labels[point_idx] = cluster_id
        
        return self
    
    def _cluster_distance(self, cluster_a, cluster_b, distance_matrix):
        """Compute distance between two clusters based on linkage method."""
        distances = []
        for i in cluster_a:
            for j in cluster_b:
                distances.append(distance_matrix[i, j])
        
        if self.linkage == 'single':
            return min(distances)  # Single linkage
        elif self.linkage == 'complete':
            return max(distances)  # Complete linkage
        elif self.linkage == 'average':
            return np.mean(distances)  # Average linkage
        else:
            raise ValueError(f"Unknown linkage: {self.linkage}")

# Apply hierarchical clustering
hierarchical = HierarchicalClustering(n_clusters=4, linkage='average')
hierarchical.fit(X_clusters)

# Compare with scikit-learn
sk_hierarchical = AgglomerativeClustering(n_clusters=4, linkage='average')
sk_hierarchical_labels = sk_hierarchical.fit_predict(X_clusters)

print(f"Our Hierarchical silhouette score: {silhouette_score(X_clusters, hierarchical.labels):.4f}")
print(f"Scikit-learn silhouette score: {silhouette_score(X_clusters, sk_hierarchical_labels):.4f}")

# Test different linkage methods
linkages = ['single', 'complete', 'average']
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, linkage in enumerate(linkages):
    hc = HierarchicalClustering(n_clusters=4, linkage=linkage)
    hc.fit(X_clusters)
    
    scatter = axes[i].scatter(X_clusters[:, 0], X_clusters[:, 1], c=hc.labels, 
                              cmap='viridis', alpha=0.6, s=50)
    axes[i].set_title(f'Hierarchical Clustering\n{linkage.capitalize()} Linkage')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Cell Analysis:**

**Linkage Methods:**
- **Single:** Closest points between clusters (can create long chains)
- **Complete:** Furthest points between clusters (creates compact clusters)
- **Average:** Mean distance between all point pairs

**Hierarchical Advantages:**
- **No need to specify K initially:** Can cut tree at different levels
- **Dendrogram:** Visual representation of clustering hierarchy
- **Deterministic:** Same results every time (unlike K-Means)

**Healthcare Analogy:** Like building a taxonomy of diseases - from specific symptoms to broad categories.

**Reflection Question:** When would you prefer hierarchical clustering over K-Means?

## Method 5: Cluster Evaluation - Silhouette and Elbow Methods

How do we know if our clustering is good?

- **Silhouette Score:** Measures how similar points are to their cluster vs other clusters
- **Elbow Method:** Finds optimal K by looking for "bend" in within-cluster distances

In [ ]:
def silhouette_score_custom(X, labels):
    """
    Compute average silhouette score for clustering quality.
    """
    n_samples = X.shape[0]
    distance_matrix = compute_distance_matrix(X)
    
    silhouette_scores = []
    
    for i in range(n_samples):
        # Points in same cluster as i
        same_cluster = labels == labels[i]
        same_cluster[i] = False  # Exclude self
        
        if np.sum(same_cluster) == 0:
            continue  # Skip isolated points
        
        # Average distance to points in same cluster (cohesion)
        a_i = np.mean(distance_matrix[i, same_cluster])
        
        # Average distance to points in other clusters (separation)
        b_i_values = []
        for cluster in np.unique(labels):
            if cluster != labels[i]:
                other_cluster = labels == cluster
                if np.sum(other_cluster) > 0:
                    b_i_values.append(np.mean(distance_matrix[i, other_cluster]))
        
        b_i = min(b_i_values) if b_i_values else a_i
        
        # Silhouette score for point i
        s_i = (b_i - a_i) / max(a_i, b_i)
        silhouette_scores.append(s_i)
    
    return np.mean(silhouette_scores) if silhouette_scores else 0

def elbow_method(X, max_k=10):
    """Find optimal K using elbow method (within-cluster sum of squares)."""
    wcss = []  # Within-cluster sum of squares
    
    for k in range(1, max_k + 1):
        kmeans_temp = KMeansClustering(n_clusters=k, random_state=42)
        kmeans_temp.fit(X)
        
        # Calculate WCSS
        wcss_k = 0
        for cluster in range(k):
            cluster_points = X[kmeans_temp.labels == cluster]
            if len(cluster_points) > 0:
                # Sum of squared distances to cluster center
                distances = [euclidean_distance(point, kmeans_temp.centers[cluster]) 
                           for point in cluster_points]
                wcss_k += np.sum(np.square(distances))
        
        wcss.append(wcss_k)
    
    return wcss

# Evaluate clustering quality
silhouette_ours = silhouette_score_custom(X_clusters, kmeans.labels)
silhouette_sk = silhouette_score(X_clusters, sk_labels)

print(f"Silhouette Scores:")
print(f"Our K-Means: {silhouette_ours:.4f}")
print(f"Scikit-learn: {silhouette_sk:.4f}")

# Elbow method for optimal K
wcss_values = elbow_method(X_clusters, max_k=8)

# Plot elbow curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Elbow plot
k_values = range(1, 9)
ax1.plot(k_values, wcss_values, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Number of Clusters (K)')
ax1.set_ylabel('Within-Cluster Sum of Squares (WCSS)')
ax1.set_title('Elbow Method: Finding Optimal K')
ax1.grid(True, alpha=0.3)
ax1.axvline(x=4, color='red', linestyle='--', alpha=0.7, label='True K=4')
ax1.legend()

# Silhouette scores for different K
silhouette_scores = []
for k in range(2, 9):
    kmeans_temp = KMeansClustering(n_clusters=k, random_state=42)
    kmeans_temp.fit(X_clusters)
    sil_score = silhouette_score_custom(X_clusters, kmeans_temp.labels)
    silhouette_scores.append(sil_score)

ax2.plot(range(2, 9), silhouette_scores, 'go-', linewidth=2, markersize=8)
ax2.set_xlabel('Number of Clusters (K)')
ax2.set_ylabel('Average Silhouette Score')
ax2.set_title('Silhouette Analysis: Cluster Quality')
ax2.grid(True, alpha=0.3)
ax2.axvline(x=4, color='red', linestyle='--', alpha=0.7, label='True K=4')
ax2.legend()

plt.tight_layout()
plt.show()

# Find optimal K from elbow (where curve starts to flatten)
optimal_k_elbow = 4  # Visually determined from plot
optimal_k_silhouette = np.argmax(silhouette_scores) + 2  # +2 because we start from k=2

print(f"\nOptimal K suggestions:")
print(f"Elbow method: K={optimal_k_elbow}")
print(f"Silhouette method: K={optimal_k_silhouette}")
print(f"True number of clusters: K=4")

**Cell Analysis:**

**Evaluation Metrics:**
- **Silhouette Score:** Higher = better separated clusters (range: -1 to 1)
- **WCSS:** Lower = tighter clusters, but decreases with more clusters
- **Elbow Point:** Where adding clusters gives diminishing returns

**Choosing K:**
- **Elbow method:** Look for sharp bend in WCSS curve
- **Silhouette:** Look for peak values
- **Domain knowledge:** Sometimes we know the "right" number of groups

**Healthcare Analogy:** Like validating disease categories - are malaria and typhoid truly distinct groups?

**Reflection Question:** Why is it important to evaluate clustering when we don't have ground truth labels?

## 🎯 Key Takeaways and Nigerian Healthcare Applications

**Algorithm Summary:**
- **KNN:** Instance-based learning using proximity
- **K-Means:** Centroid-based clustering with Lloyd's algorithm
- **Hierarchical:** Tree-based clustering with different linkage methods
- **Evaluation:** Silhouette scores and elbow method for quality assessment

**Healthcare Translation - Mark:**

Imagine building AI for Nigerian hospitals:
- **KNN Diagnosis:** "This patient is most similar to 3 malaria cases and 1 typhoid case"
- **Patient Clustering:** Group patients by symptom patterns to discover new disease subtypes
- **Hierarchical Categories:** From specific symptoms (fever+headache) to broad conditions (infectious diseases)
- **Quality Assessment:** Ensure patient groups are meaningful, not random

**Performance achieved:** Our implementations approach industry standards!

**Reflection Questions:**
1. How would you use clustering to discover unknown disease patterns in Nigerian health data?
2. When should you prefer KNN over other classification methods for medical diagnosis?
3. How might distance metrics affect clustering of patients from different Nigerian regions?

**Next Steps:**
- Add support for different distance metrics in clustering
- Implement K-Means++ initialization for better convergence
- Explore density-based clustering (DBSCAN)

**🏆 Exceptional work, my student! You've mastered proximity-based learning from the ground up.**